# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 30 2022 8:30AM,247598,10,0086019029,ISDIN Corporation,Released
1,Sep 30 2022 8:30AM,247598,10,0086020198,ISDIN Corporation,Released
2,Sep 30 2022 8:30AM,247598,10,0086023326,ISDIN Corporation,Released
3,Sep 30 2022 8:30AM,247598,10,0086023378,ISDIN Corporation,Released
4,Sep 30 2022 8:30AM,247598,10,0086023719,ISDIN Corporation,Released
5,Sep 30 2022 8:30AM,247598,10,0086023730,ISDIN Corporation,Released
6,Sep 30 2022 8:30AM,247598,10,0086023746,ISDIN Corporation,Released
7,Sep 30 2022 8:30AM,247598,10,0086023755,ISDIN Corporation,Released
8,Sep 30 2022 8:30AM,247598,10,0086023764,ISDIN Corporation,Released
9,Sep 30 2022 8:30AM,247598,10,0086023784,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,247600,Executing,5
16,247600,Released,1
17,247601,Released,6
18,247602,Released,1
19,247604,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247599,NaN,NaN,1.0
247600,NaN,5.0,1.0
247601,NaN,NaN,6.0
247602,NaN,NaN,1.0
247604,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247396,0.0,0.0,1.0
247527,11.0,5.0,0.0
247534,0.0,0.0,1.0
247558,3.0,1.0,0.0
247576,0.0,0.0,31.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247396,0,0,1
247527,11,5,0
247534,0,0,1
247558,3,1,0
247576,0,0,31


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247396,0,0,1
1,247527,11,5,0
2,247534,0,0,1
3,247558,3,1,0
4,247576,0,0,31


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247396,,,1
1,247527,11,5,
2,247534,,,1
3,247558,3,1,
4,247576,,,31


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation
21,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC"
22,Sep 30 2022 8:30AM,247600,10,"Methapharm, Inc."
28,Sep 30 2022 8:30AM,247601,10,ISDIN Corporation
34,Sep 30 2022 8:30AM,247604,10,ISDIN Corporation
35,Sep 29 2022 4:20PM,247602,19,ACG North America LLC
36,Sep 29 2022 3:21PM,247595,22,"NBTY Global, Inc."
37,Sep 29 2022 3:20PM,247594,22,"NBTY Global, Inc."
38,Sep 29 2022 3:20PM,247593,22,"NBTY Global, Inc."
39,Sep 29 2022 2:22PM,247396,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation,,,21
1,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC",,,1
2,Sep 30 2022 8:30AM,247600,10,"Methapharm, Inc.",,5,1
3,Sep 30 2022 8:30AM,247601,10,ISDIN Corporation,,,6
4,Sep 30 2022 8:30AM,247604,10,ISDIN Corporation,,,1
5,Sep 29 2022 4:20PM,247602,19,ACG North America LLC,,,1
6,Sep 29 2022 3:21PM,247595,22,"NBTY Global, Inc.",,,1
7,Sep 29 2022 3:20PM,247594,22,"NBTY Global, Inc.",,,1
8,Sep 29 2022 3:20PM,247593,22,"NBTY Global, Inc.",,,1
9,Sep 29 2022 2:22PM,247396,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation,21,,
1,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC",1,,
2,Sep 30 2022 8:30AM,247600,10,"Methapharm, Inc.",1,5,
3,Sep 30 2022 8:30AM,247601,10,ISDIN Corporation,6,,
4,Sep 30 2022 8:30AM,247604,10,ISDIN Corporation,1,,
5,Sep 29 2022 4:20PM,247602,19,ACG North America LLC,1,,
6,Sep 29 2022 3:21PM,247595,22,"NBTY Global, Inc.",1,,
7,Sep 29 2022 3:20PM,247594,22,"NBTY Global, Inc.",1,,
8,Sep 29 2022 3:20PM,247593,22,"NBTY Global, Inc.",1,,
9,Sep 29 2022 2:22PM,247396,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation,21,,
1,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC",1,,
2,Sep 30 2022 8:30AM,247600,10,"Methapharm, Inc.",1,5,
3,Sep 30 2022 8:30AM,247601,10,ISDIN Corporation,6,,
4,Sep 30 2022 8:30AM,247604,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation,21.0,NaN,NaN
1,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
2,Sep 30 2022 8:30AM,247600,10,"Methapharm, Inc.",1.0,5.0,NaN
3,Sep 30 2022 8:30AM,247601,10,ISDIN Corporation,6.0,NaN,NaN
4,Sep 30 2022 8:30AM,247604,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 30 2022 8:30AM,247598,10,ISDIN Corporation,21.0,0.0,0.0
1,Sep 30 2022 8:30AM,247599,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
2,Sep 30 2022 8:30AM,247600,10,"Methapharm, Inc.",1.0,5.0,0.0
3,Sep 30 2022 8:30AM,247601,10,ISDIN Corporation,6.0,0.0,0.0
4,Sep 30 2022 8:30AM,247604,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1980683,67.0,13.0,13.0
16,495092,1.0,1.0,3.0
19,247602,1.0,0.0,0.0
21,247396,1.0,0.0,0.0
22,742782,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1980683,67.0,13.0,13.0
1,16,495092,1.0,1.0,3.0
2,19,247602,1.0,0.0,0.0
3,21,247396,1.0,0.0,0.0
4,22,742782,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,67.0,13.0,13.0
1,16,1.0,1.0,3.0
2,19,1.0,0.0,0.0
3,21,1.0,0.0,0.0
4,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,67.0
1,16,Released,1.0
2,19,Released,1.0
3,21,Released,1.0
4,22,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21,22
Status,,,,,
Completed,13.0,3.0,0.0,0.0,0.0
Executing,13.0,1.0,0.0,0.0,0.0
Released,67.0,1.0,1.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21,22
0,Completed,13.0,3.0,0.0,0.0,0.0
1,Executing,13.0,1.0,0.0,0.0,0.0
2,Released,67.0,1.0,1.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21,22
0,Completed,13.0,3.0,0.0,0.0,0.0
1,Executing,13.0,1.0,0.0,0.0,0.0
2,Released,67.0,1.0,1.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()